In [1]:
import os
import shutil

from azureml.core.workspace import Workspace
from azureml.core.environment import Environment
from azureml.core import Experiment, ScriptRunConfig
from azureml.train.estimator import Estimator

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.dnn import PyTorch
from azureml.core.conda_dependencies import CondaDependencies
ws = Workspace.from_config()
env = Environment.get(workspace=ws, name="AzureML-PyTorch-1.1-GPU")
print(env)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code APZSEJ2CY to authenticate.
Interactive authentication successfully completed.
Environment(Name: AzureML-PyTorch-1.1-GPU,
Version: 12)


In [2]:
project_folder = './train-main'
os.makedirs(project_folder, exist_ok=True)

experiment_name = 'synbio-train3'
experiment = Experiment(name=experiment_name,workspace = ws)

In [3]:
cluster_name = "gpu-traindlarge"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
    
except ComputeTargetException:
    print('Compute target not found')

print(compute_target)

Found existing compute target
AmlCompute(workspace=Workspace.create(name='ml-main', subscription_id='f1be6293-6d1c-4e77-bb06-b2083e9aeb20', resource_group='synbiolic-main'), name=gpu-traindlarge, id=/subscriptions/f1be6293-6d1c-4e77-bb06-b2083e9aeb20/resourceGroups/synbiolic-main/providers/Microsoft.MachineLearningServices/workspaces/ml-main/computes/gpu-traindlarge, type=AmlCompute, provisioning_state=Succeeded, location=eastus, tags=None)


In [4]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_GPU_IMAGE

# Create a new runconfig object
run_amlcompute = RunConfiguration()

# Use the cpu_cluster you created above. 
run_amlcompute.target = cluster_name

# Enable Docker
run_amlcompute.environment.docker.enabled = True

# Set Docker base image to the default CPU-based image
run_amlcompute.environment.docker.base_image = DEFAULT_GPU_IMAGE

# Use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_amlcompute.environment.python.user_managed_dependencies = False

# Specify CondaDependencies obj, add necessary packages
run_amlcompute.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [5]:
estimator = PyTorch(source_directory=project_folder,
                     compute_target=compute_target,
                     conda_dependencies_file_path='export_noversion.yml',
                     entry_script='train.py',
                     use_gpu=True)

WARNING - If environment_definition or conda_dependencies_file_path is specified, Azure ML will not install any framework related packages on behalf of the user.
WARNING - 'conda_dependencies_file_path' parameter will be deprecated. Please use 'conda_dependencies_file' instead
WARNING - framework_version is not specified, defaulting to version 1.3.


In [6]:
run = experiment.submit(estimator)

In [7]:
run.wait_for_completion(show_output=True)

RunId: synbio-train3_1583872641_3c5d3e9f
Web View: https://ml.azure.com/experiments/synbio-train3/runs/synbio-train3_1583872641_3c5d3e9f?wsid=/subscriptions/f1be6293-6d1c-4e77-bb06-b2083e9aeb20/resourcegroups/synbiolic-main/workspaces/ml-main

Streaming azureml-logs/20_image_build_log.txt

2020/03/10 20:37:58 Downloading source code...
2020/03/10 20:38:00 Finished downloading source code
2020/03/10 20:38:01 Creating Docker network: acb_default_network, driver: 'bridge'
2020/03/10 20:38:01 Successfully set up Docker network: acb_default_network
2020/03/10 20:38:01 Setting up Docker configuration...
2020/03/10 20:38:02 Successfully set up Docker configuration
2020/03/10 20:38:02 Logging in to registry: mlmain2c9169d7.azurecr.io
2020/03/10 20:38:04 Successfully logged into mlmain2c9169d7.azurecr.io
2020/03/10 20:38:04 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/03/10 20:38:04 Scanning for dependencies...
2020/03/10 20:38:07

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Image build failed. For more details, check log file azureml-logs/20_image_build_log.txt",
        "details": []
    },
    "correlation": {
        "operation": null,
        "request": "3829006853f6cbe0"
    },
    "environment": "eastus",
    "location": "eastus",
    "time": "2020-03-10T22:08:16.232375Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Image build failed. For more details, check log file azureml-logs/20_image_build_log.txt\",\n        \"details\": []\n    },\n    \"correlation\": {\n        \"operation\": null,\n        \"request\": \"3829006853f6cbe0\"\n    },\n    \"environment\": \"eastus\",\n    \"location\": \"eastus\",\n    \"time\": \"2020-03-10T22:08:16.232375Z\"\n}"
    }
}